## Setup

In [7]:
# Import Libraries
import time
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

## Delete collection before inserting new data

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.Stocks_db.cash_flow.drop()

## CBA Cashflow statement table

In [9]:
############# Extract #############
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

bank='CBA'
url = f'https://au.finance.yahoo.com/quote/{bank}.AX/cash-flow?p={bank}.AX'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
cashflow_table_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
cashflow_table_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in cashflow_table_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in cashflow_table_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
cashflow_table = pd.DataFrame(table_rows, columns=table_headers_list)
############# Extract #############

############# Transform #############
# Remove all the rows without values based on any of the financial year columns
cashflow_table_new = cashflow_table[cashflow_table['29/06/2021']!=''].copy()
# Replace all fields containing '-' with 0
cashflow_table_new = cashflow_table_new.replace('-',0)
# Set the index to Breakdown
cashflow_table_new = cashflow_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
cashflow_table_new_17to18 = cashflow_table_new[['29/06/2018']].copy()
#2018-2019
cashflow_table_new_18to19 = cashflow_table_new[['29/06/2019']].copy()
#2019-2020
cashflow_table_new_19to20 = cashflow_table_new[['29/06/2020']].copy()
#2020-2021
cashflow_table_new_20to21 = cashflow_table_new[['29/06/2021']].copy()
# TTM
cashflow_table_new_ttm = cashflow_table_new[['ttm']].copy()
############# Transform #############

############# Load #############
# Load Into Mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

cba_dict = {}

cba_cash_flow_reports = {'29/06/2018':cashflow_table_new_17to18.to_dict()['29/06/2018'],
                         '29/06/2019':cashflow_table_new_18to19.to_dict()['29/06/2019'],
                         '29/06/2020':cashflow_table_new_19to20.to_dict()['29/06/2020'],
                         '29/06/2021':cashflow_table_new_20to21.to_dict()['29/06/2021'],
                         'ttm':cashflow_table_new_ttm.to_dict()['ttm']}


cba_dict["CBA"] = cba_cash_flow_reports

client.Stocks_db.cash_flow.insert_one(cba_dict)
############# Load #############

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Cam\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


In [10]:
############# Extract #############
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

bank='CBA'
url = f'https://au.finance.yahoo.com/quote/{bank}.AX/cash-flow?p={bank}.AX'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
cashflow_table_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
cashflow_table_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in cashflow_table_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in cashflow_table_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
cashflow_table = pd.DataFrame(table_rows, columns=table_headers_list)
cashflow_table
############# Extract #############

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Cam\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


,Breakdown,ttm,29/06/2021,29/06/2020,29/06/2019,29/06/2018
0,Cash flows from operating activities,,,,,
1,Net income,"10,181,000","10,181,000","9,634,000","8,571,000","9,329,000"
2,Other working capital,"40,545,000","40,545,000","37,268,000","17,446,000","129,000"
3,Net cash provided by operating activities,"41,312,000","41,312,000","38,860,000","18,086,000","1,109,000"
4,Cash flows from investing activities,,,,,
5,"Investments in property, plant and equipment","-767,000","-767,000","-1,592,000","-640,000","-980,000"
6,"Acquisitions, net","-121,000","-121,000","-18,000",-,"-271,000"
7,Purchases of investments,-,-,"-18,000","72,000","-271,000"
8,Net cash used for investing activities,"871,000","871,000","3,696,000","983,000","-1,002,000"
9,Cash flows from financing activities,,,,,


In [11]:
############# Transform #############
# Remove all the rows without values based on any of the financial year columns
cashflow_table_new = cashflow_table[cashflow_table['29/06/2021']!=''].copy()
# Replace all fields containing '-' with 0
cashflow_table_new = cashflow_table_new.replace('-',0)
# Set the index to Breakdown
cashflow_table_new = cashflow_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
cashflow_table_new_17to18 = cashflow_table_new[['29/06/2018']].copy()
#2018-2019
cashflow_table_new_18to19 = cashflow_table_new[['29/06/2019']].copy()
#2019-2020
cashflow_table_new_19to20 = cashflow_table_new[['29/06/2020']].copy()
#2020-2021
cashflow_table_new_20to21 = cashflow_table_new[['29/06/2021']].copy()
# TTM
cashflow_table_new_ttm = cashflow_table_new[['ttm']].copy()
############# Transform #############

In [12]:
cashflow_table_new_17to18

,29/06/2018
Breakdown,
Net income,"9,329,000"
Other working capital,"129,000"
Net cash provided by operating activities,"1,109,000"
"Investments in property, plant and equipment","-980,000"
"Acquisitions, net","-271,000"
Purchases of investments,"-271,000"
Net cash used for investing activities,"-1,002,000"
Debt repayment,"-68,273,000"
Common stock issued,"55,000"


In [13]:
############# Load #############
# Load Into Mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

print(cashflow_table_new_17to18.to_dict()['29/06/2018'])

cba_cash_flow_reports = {'29/06/2018':cashflow_table_new_17to18.to_dict()['29/06/2018'],
                         '29/06/2019':cashflow_table_new_18to19.to_dict()['29/06/2019'],
                         '29/06/2020':cashflow_table_new_19to20.to_dict()['29/06/2020'],
                         '29/06/2021':cashflow_table_new_20to21.to_dict()['29/06/2021'],
                         'ttm':cashflow_table_new_ttm.to_dict()['ttm']}



cba_dict["CBA"] = cba_cash_flow_reports

cba_dict

client.Stocks_db.cash_flow.insert_one(cba_dict)
############# Load #############

{'Net income': '9,329,000', 'Other working capital': '129,000', 'Net cash provided by operating activities': '1,109,000', 'Investments in property, plant and equipment': '-980,000', 'Acquisitions, net': '-271,000', 'Purchases of investments': '-271,000', 'Net cash used for investing activities': '-1,002,000', 'Debt repayment': '-68,273,000', 'Common stock issued': '55,000', 'Common stock repurchased': '-95,000', 'Dividends paid': '-5,366,000', 'Other financing activities': '27,000', 'Net cash used provided by (used for) financing activities': '-934,000', 'Net change in cash': '-827,000', 'Cash at beginning of period': '23,117,000', 'Cash at end of period': '23,005,000', 'Operating cash flow': '1,109,000', 'Capital expenditure': '-980,000', 'Free cash flow': '129,000'}


DuplicateKeyError: E11000 duplicate key error collection: Stocks_db.cash_flow index: _id_ dup key: { _id: ObjectId('6178244413735aab046ec25f') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6178244413735aab046ec25f')}, 'errmsg': "E11000 duplicate key error collection: Stocks_db.cash_flow index: _id_ dup key: { _id: ObjectId('6178244413735aab046ec25f') }"}

## ANZ Cashflow statement table

In [17]:
############# Extract #############
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

bank='ANZ'
url = f'https://au.finance.yahoo.com/quote/{bank}.AX/cash-flow?p={bank}.AX'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
cashflow_table_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
cashflow_table_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in cashflow_table_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in cashflow_table_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
cashflow_table = pd.DataFrame(table_rows, columns=table_headers_list)

############# Extract #############

############# Transform #############
# Remove all the rows without values based on any of the financial year columns
cashflow_table_new = cashflow_table[cashflow_table['29/09/2020']!=''].copy()
# Replace all fields containing '-' with 0
cashflow_table_new = cashflow_table_new.replace('-',0)
# Set the index to Breakdown
cashflow_table_new = cashflow_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
cashflow_table_new_17to18 = cashflow_table_new[['29/09/2018']].copy()
#2018-2019
cashflow_table_new_18to19 = cashflow_table_new[['29/09/2019']].copy()
#2019-2020
cashflow_table_new_19to20 = cashflow_table_new[['29/09/2020']].copy()
# TTM
cashflow_table_new_ttm = cashflow_table_new[['ttm']].copy()
############# Transform #############

############# Load #############
# Load Into Mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

anz_dict = {}

anz_cash_flow_reports = {'29/09/2018':cashflow_table_new_17to18.to_dict()['29/09/2018'],
                         '29/09/2019':cashflow_table_new_18to19.to_dict()['29/09/2019'],
                         '29/09/2020':cashflow_table_new_19to20.to_dict()['29/09/2020'],
                         'ttm':cashflow_table_new_ttm.to_dict()['ttm']}


anz_dict["ANZ"] = anz_cash_flow_reports

client.Stocks_db.cash_flow.insert_one(anz_dict)
############# Load #############

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\James\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


In [18]:
cashflow_table

,Breakdown,ttm,29/09/2020,29/09/2019,29/09/2018
0,Cash flows from operating activities,,,,
1,Net income,"4,975,000","3,577,000","5,953,000","6,400,000"
2,Depreciation & amortisation,"1,341,000","1,391,000","871,000","1,199,000"
3,Change in working capital,"31,623,000","46,794,000","-16,625,000","-4,498,000"
4,Other working capital,"27,458,000","52,284,000","-4,550,000","10,566,000"
5,Other non-cash items,"93,000","-80,000","-356,000","-55,000"
6,Net cash provided by operating activities,"27,458,000","52,284,000","-4,550,000","10,566,000"
7,Cash flows from investing activities,,,,
8,Purchases of investments,"-35,523,000","-40,829,000","-23,847,000","-23,806,000"
9,Sales/maturities of investments,"22,599,000","29,951,000","24,149,000","22,740,000"
